#### Importing libraries 

In [ ]:
import cv2
import face_recognition as fr
import numpy
import os
from datetime import datetime

#### Import the test images and store the names of the people in a list 

In [ ]:
path=r"G:\openCV_project\Image Basics"
images=[]
classNames=[]
mylist=os.listdir(path)
print(mylist)

for cl in mylist:
    curImg=cv2.imread(f'{path}/{cl}')
    images.append(curImg)
    classNames.append(os.path.splitext(cl)[0])
print(classNames)

#### A function findEncodings() is written to find the encodings of the images using using face_encodings() which is a function of face_recognition library.

In [ ]:
def findEncodings(images):
    encodeList=[]
    for img in images:
        img=cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
        encode=fr.face_encodings(img)[0]
        encodeList.append(encode)
    return encodeList

#### A function markAttendance() is written to save the name and joining time of the person in a CSV file. If a name is present in the file, that name would not be saved again.

In [ ]:
def markAttendance(name):
    with open(r"G:\openCV_project\attendance.csv","r+") as f:
        myDataList=f.readlines()
        nameList=[]
        print(myDataList)
        for line in myDataList:
            entry=line.split(',')
            nameList.append(entry[0])
        if name not in nameList:
            now=datetime.now()
            dtString=now.strftime("%H:%M:%S")
            f.writelines(f'\n{name},{dtString}')

#### Calling findEncodings() to determine the face encoding of the images present in the image folder.

In [ ]:
encodeListKnown=findEncodings(images)
print("encoding completes")

#### Now, we will execute the face recognition function on the faces taken through live webcam feed and record the encodings of those recognised images.

#### These encodings will be matched with the recorded encodings of the images present in the folder and distance is find between the encodings using face_distance() present in face_recognition module. The image with lowest distance would be the best match and that name will be displayed over the image of live webcam feed. The name and current time of the matched image is saved in CSV file using markAttendance().

In [ ]:
cap=cv2.VideoCapture(0)
while True:
    success,img=cap.read()
    #reduce size of image to speed up the process
    imgS=cv2.resize(img,(0,0),None,0.25,0.25)
    imgS=cv2.cvtColor(imgS,cv2.COLOR_BGR2RGB)
    facesCurFrame=fr.face_locations(imgS)
    encodesCurFrame=fr.face_encodings(imgS,facesCurFrame)
    
    for encodeFace,faceLoc in zip(encodesCurFrame,facesCurFrame):
        matches=fr.compare_faces(encodeListKnown,encodeFace)
        faceDis=fr.face_distance(encodeListKnown,encodeFace)
        print(faceDis)
        
        #find the best match with lowest distance
        matchIndex=numpy.argmin(faceDis)
        if matches[matchIndex]:
            name=classNames[matchIndex].upper()
            print(name)
            
            #putting the text and rectangle on the image recognised in webcam feed
            y1,x2,y2,x1=faceLoc
            y1,x2,y2,x1=y1*4,x2*4,y2*4,x1*4
            cv2.rectangle(img,(x1,y1),(x2,y2),(0,255,0),2)
            cv2.rectangle(img,(x1,y2-35),(x2,y2),(0,255,0),cv2.FILLED)
            cv2.putText(img,name,(x1+6,y2-6),cv2.FONT_HERSHEY_COMPLEX,1,(255,255,255),2)
            markAttendance(name)
            
    cv2.imshow("webcam",img)
    if cv2.waitKey()==ord("s"):
        break
cv2.destroyAllWindows()
cap.release()
